In [1]:
%cd ..

/home/daryna/pet_projects/ring-customiser-chat-bot


/home/daryna/miniconda3/envs/ring-customizer/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import json
from dotenv import load_dotenv

from pathlib import Path
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import Docx2txtLoader, TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory, ConversationBufferMemory, ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import ValidationError

from src.schemas import Ring

load_dotenv()

True

In [3]:
DATA_DIR = Path('data')
customization_filepath = DATA_DIR / 'customization.md'
faq_filepath = DATA_DIR / 'FAQ.md'

customization_loader = TextLoader(file_path=customization_filepath, encoding='utf-8')
faq_loader = TextLoader(file_path=faq_filepath, encoding='utf-8')

customization_documents = customization_loader.load()
faq_documents = faq_loader.load();

In [4]:
embedding_model = OpenAIEmbeddings()
index = VectorstoreIndexCreator(
    embedding=embedding_model,
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([customization_loader, faq_loader])

/home/daryna/miniconda3/envs/ring-customizer/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/home/daryna/miniconda3/envs/ring-customizer/lib/python3.11/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [5]:
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)

In [6]:
query ="Please list all possible sizes for the ring"
response = index.query(query, llm=llm)
display(response)


'We offer American sizes from 4 to 13, including half sizes. Here’s a table mapping American sizes to millimeters:\n| American Size | Diameter (mm) |\n|---------------|---------------|\n| 4             | 14.9          |\n| 4.5           | 15.3          |\n| 5             | 15.7          |\n| 5.5           | 16.1          |\n| 6             | 16.5          |\n| 6.5           | 16.9          |\n| 7             | 17.3          |\n| 7.5           | 17.7          |\n| 8             | 18.1          |\n| 8.5           | 18.5          |\n| 9             | 18.9          |\n| 9.5           | 19.3          |\n| 10            | 19.7          |\n| 10.5          | 20.1          |\n| 11            | 20.5          |\n| 11.5          | 20.9          |\n| 12            | 21.3          |\n| 12.5          | 21.7          |\n| 13            | 22.1          |'

In [7]:
query ="What is my American size of the ring, if it is 15.1mm in diameter?"
response = index.query(query, llm=llm)
display(response)

'If the diameter of the ring is 15.1mm, the corresponding American size would be between 4.5 and 5. Since the diameter falls between 15.3mm for size 4.5 and 15.7mm for size 5, the closest size would be 5.'

In [8]:
memory = ConversationBufferWindowMemory(k=10, ai_prefix='AI', human_prefix='Customer')
with open(customization_filepath, 'r') as f:
    customization_text = f.read()


template = f"""
AI is interacting with the Customer to collect ring customization preferences.
AI is listing all the possible customizations for the ring in the first message to the Customer, which are:
{customization_text}
And asks the Customer to specify selected customizations. If the Customer missed something the AI
asks about the missing customization. The Customer should provide all the necessary information without leaving any details out. 
The AI, on the other hand, will not make assumptions or provide answers on behalf of the Customer.
"""
template += """
Then the AI collects the Customer's answers and fill up the output JSON with the following fields:
Material
Style
Surface
Size
Ring width
Engraving

In the Customer does not want engraving set the field Engraving to null.
After all customizations are collected AI asks the Customer if everything is correct. In case of 
confirmation from the Customer is received the AI should output JSON (and only JSON) above with field Verified: true 
added to. In case of verified false ask the Customer about correctness again.

Current conversation:
{history}
Customer: {input}
AI:
"""

prompt_template = PromptTemplate(
    input_variables=['history', 'input'], 
    template=template
)

conversation = ConversationChain(
    prompt=prompt_template,
    llm=llm,
    memory=memory,
    verbose=False
)

In [9]:
from typing import Optional


def attemp_ring_parsing(response) -> Optional[Ring]:
    try:
        ring_dict = json.loads(response)
        try:
            if not ring_dict.get('Verified', None):
                not_verified_input = """
Field Verified is missing or set to false. 
The AI should ask the Customer about the correctness of the ring customizations, 
showing the current customizations.
                """
                print('\nAdmin:', not_verified_input)
                response = conversation.predict(
                    input=not_verified_input
                )
                print('\nAI:', response)
            else:
                ring = Ring(**{k.lower().replace(' ', '_'): v for k, v in ring_dict.items()})
                return ring
        except ValidationError as e:
            fix_input = f"""
When parsing the output the following error happened: {e}.
Continue the conversation, explain and ask the Customer about the issue and clarify
the order, to fix the issue. AI does not attempt to change the customizations instead of the Customer,
but is being helpful in suggesting what can be changed for the order to be accepted.
After the issues above are solved AI asks the user about correctness of the selected customizations 
one more time.
"""
            print('\nAdmin:', fix_input)
            response = conversation.predict(
                input=fix_input
            )
            print('\nAI:', response)

    except json.decoder.JSONDecodeError:
        return None

def handle_conversation(conversation):
    manual_input = input()
    print('\nCustomer:', manual_input)
    response = conversation.predict(
        input=manual_input
    )
    print('\nAI:', response)
    

In [10]:
while True:
    manual_input = ''
    while manual_input == '':
        manual_input = input()

    print('\nCustomer:', manual_input)
    response = conversation.predict(
        input=manual_input
    )
    print('\nAI:', response)
    if ring := attemp_ring_parsing(response):
        break


Customer: hello
AI: Hello! I'm here to help you customize your ring. Here are the possible customizations you can choose from:
Material:
- Yellow Gold
- White Gold
- Platinum
- Sterling Silver
- Titanium

Style:
- Classic
- Modern
- Vintage
- Bohemian

Surface:
- Polished
- Matte
- Hammered
- Brushed

Size: From 4 to 13

Ring Width: From 1mm to 8mm

Engraving: Up to 20 characters or empty

Please specify your selected customizations. If you missed anything, feel free to let me know.
Customer: gold
AI: I see you've selected Gold as the material. Could you please specify the style, surface, size, ring width, and engraving preferences as well?
Customer: polished
AI: Great choice! So far we have:
Material: Gold
Surface: Polished

Could you please specify the style, size, ring width, and engraving preferences as well?
Customer: what are possible styles again?
AI: Here are the possible styles:
- Classic
- Modern
- Vintage
- Bohemian

Please let me know your preferred style, size, ring width,

In [11]:
ring

Ring(material='White Gold', style='Bohemian', surface='Polished', size=5.0, ring_width=2.0, engraving=None)